# Construindo um vector database em um banco de dados relacional

Nesse notebook iremos inserir um vector database em um banco de dados relacional.

O objetivo é demonstrar que podemos inserir os vetores e fazer queries com eles também.


## 1 - Configuração

In [1]:
# Recarrega automaticamente quando há modificação na pasta src/
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import sqlite3
from pathlib import Path

In [3]:
# Configuração de diretórios
HERE = os.path.abspath(".")
MODULES = Path(HERE).parent
DATA = Path(HERE).parent / "data"

sys.path.insert(
    0, str(MODULES)
)  # para ser possível importar os módulos personalizados da pasta src/

In [4]:
from src.io import SQLiteIO

sqlite = SQLiteIO(DATA / "2_processed" / "database-vector.db")

## 2 - Criando tabela

In [5]:
sqlite.query(
    """
        CREATE TABLE IF NOT EXISTS vectors (
            id INTEGER PRIMARY KEY,
            vector BLOB NOT NULL
        )
    """
)

## 3 - Criando vetores aleatórios

In [6]:
from src.utils import create_random_vector

vec1 = create_random_vector((0, 10), 4)
vec2 = create_random_vector((0, 10), 4)
print(f"Vector 1: {vec1} \nVector 2: {vec2}")

Vector 1: [2.22981626 8.48184276 6.50987484 2.73691434] 
Vector 2: [7.95405356 5.40855442 5.02539578 1.09566433]


## 4 - Populando tabela com os vetores

In [7]:
from src.utils import transform_np_array

sqlite.query(
    "INSERT INTO vectors (vector) VALUES(?)",
    (sqlite3.Binary(transform_np_array(vec1, "bytes")),),
)
sqlite.query(
    "INSERT INTO vectors (vector) VALUES(?)",
    (sqlite3.Binary(transform_np_array(vec2, "bytes")),),
)

sqlite.close()

In [8]:
sqlite.read(table_name="vectors")

,id,vector
0,1,"b'\xc6Gz\xe8\xa9\xd6\x01@:\xd2,\x18\xb4\xf6 @\..."
1,2,b'hM\xdfj\xf3\xd0\x1f@\x01N\x1b\x17\\\xa2\x15@...


## 5 - Buscando o vetor mais próximo da query

In [9]:
vector_query = transform_np_array(create_random_vector((0, 10), 4), "bytes")

result = sqlite.query(
    f"SELECT vector FROM vectors ORDER BY abs(vector - ?) ASC",
    (sqlite3.Binary(vector_query),),
)
row = result.fetchone()
print(row)

(b'\xc6Gz\xe8\xa9\xd6\x01@:\xd2,\x18\xb4\xf6 @\xaf\xae\\\xa1\x1c\n\x1a@t\x86\x9fX3\xe5\x05@',)


In [10]:
from src.utils import desserialize_bytes_array

print(desserialize_bytes_array(row[0]))

[2.22981626 8.48184276 6.50987484 2.73691434]
